In [1]:

import numpy as np
import tensorflow as tf
from tqdm import tqdm, trange
from tqdm.keras import TqdmCallback
import argparse

import models
import data_utils

import sys; sys.path.append('../'); sys.path.append('../..')
import utils

2024-01-27 01:09:44.365573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-27 01:09:44.710777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 01:09:47.720147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
args = argparse.Namespace(model='relconv_groupattn', normalizer=None, freeze_embedder=False, object_selection=False, 
    task='1task_between', train_split='hexos', test_split_size=5_000, test_size=5_000, val_size=1_000, n_epochs=100, batch_size=512, 
    learning_rate=0.001, early_stopping=True, train_size=-1,
    num_trials=1, start_trial=0, wandb_project_name='tmp', seed=314159, ignore_gpu_assert=False)

In [3]:

# #region setup
# # parse script arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('--model', type=str,
#     choices=models.model_creators.keys(),
#     help='the model to evaluate learning curves on')
# parser.add_argument('--normalizer', type=str, default=None, choices=('l2', 'tcn', 'None', None))
# parser.add_argument('--freeze_embedder', type=bool, default=False)
# parser.add_argument('--object_selection', type=bool, default=False)
# parser.add_argument('--task', type=str, help='the relational games task')
# parser.add_argument('--train_split', type=str, choices=('stripes', 'hexos', 'pentos'))
# parser.add_argument('--test_split_size', type=int, default=5_000,
#     help='size of sample from hold out sets to evaluate on')
# parser.add_argument('--val_size', type=int, default=1_000)
# parser.add_argument('--test_size', type=int, default=5_000)
# parser.add_argument('--n_epochs', default=100, type=int, help='number of epochs to train each model for')
# parser.add_argument('--batch_size', default=512, help='batch size')
# parser.add_argument('--learning_rate', default=0.001, help='learning rate')
# parser.add_argument('--early_stopping', action='store_true')
# parser.add_argument('--train_size', default=-1, type=int, help='training set size')
# parser.add_argument('--num_trials', default=1, type=int, help='number of trials per training set size')
# parser.add_argument('--start_trial', default=0, type=int, help='what to call first trial')
# parser.add_argument('--wandb_project_name', default=None, type=str, help='W&B project name')
# parser.add_argument('--seed', default=314159, help='random seed')
# parser.add_argument('--ignore_gpu_assert', action='store_true')
# args = parser.parse_args()

In [4]:

utils.print_section("SET UP")

print(f'received the following arguments: {args}')

# set random seed
tf.random.set_seed(args.seed)

# check if GPU is being used
print(tf.config.list_physical_devices())
if not args.ignore_gpu_assert:
    assert len(tf.config.list_physical_devices('GPU')) > 0

# set up W&B logging
import wandb
wandb.login()

import logging
logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

wandb_project_name = args.wandb_project_name
if wandb_project_name is None:
    wandb_project_name = f'relational_games-{args.task}'

#endregion

#region load data
utils.print_section('LOADING DATA')
data_path = '../../data/relational_games'

dataset_specs = np.load(f'{data_path}/dataset_specs.npy', allow_pickle=True).item()

task_datasets = data_utils.load_task_datasets(args.task, data_path, data_format='tfrecord', ds_specs=dataset_specs)

train_split_ds = task_datasets[args.train_split]
# train_split_ds = data_utils.load_ds_from_npz(filename)

train_ds, val_ds, test_ds = utils.split_ds(train_split_ds, val_size=args.val_size, test_size=args.test_size)
del train_split_ds

batch_size = args.batch_size
val_ds = val_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

eval_datasets = {split: ds.take(args.test_split_size).batch(batch_size) for split, ds in task_datasets.items() if split != args.train_split}

#endregion



SET UP


received the following arguments: Namespace(model='relconv_groupattn', normalizer=None, freeze_embedder=False, object_selection=False, task='1task_between', train_split='hexos', test_split_size=5000, test_size=5000, val_size=1000, n_epochs=100, batch_size=512, learning_rate=0.001, early_stopping=True, train_size=-1, num_trials=1, start_trial=0, wandb_project_name='tmp', seed=314159, ignore_gpu_assert=False)
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-01-27 01:09:57.933850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:09:58.107748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:09:58.107956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: awni00. Use `wandb login --relogin` to force relogin




LOADING DATA




2024-01-27 01:10:03.758593: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:10:03.760038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:10:03.760667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:10:07.678079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-27 01:10:07.680234: I tensorflow/compile

In [5]:

#region training setup
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
create_opt = lambda: tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
# metrics = [
#         tf.keras.metrics.BinaryAccuracy(name='acc'),
#         tf.keras.metrics.Precision(class_id=1, name='precision'),
#         tf.keras.metrics.Recall(class_id=1, name='recall'),
#         tf.keras.metrics.F1Score(average='weighted', name='f1'),
#         tf.keras.metrics.AUC(curve='ROC', multi_label=True, name='auc')
#         ]
metrics = ['acc']

def create_callbacks(data_size=None, batch_size=None):
    callbacks = [
        # TqdmCallback(data_size=data_size, batch_size=batch_size),
        wandb.keras.WandbMetricsLogger(log_freq='batch')
    ]

    if args.early_stopping:
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, start_from_epoch=20))

    return callbacks

fit_kwargs = dict(epochs=args.n_epochs)
#endregion

In [6]:

# region train model
train_size = args.train_size

num_trials = args.num_trials # num of trials per train set size
start_trial = args.start_trial
# endregion

#region functions
def train_model(
    create_model, eval_model, fit_kwargs, create_callbacks,
    wandb_project_name, group_name,
    train_ds, val_ds, test_ds,
    train_size, start_trial, num_trials,
    ):

    for trial in trange(start_trial, start_trial + num_trials, desc='trial', leave=False):
        run = wandb.init(project=wandb_project_name, group=group_name, name=f'trial = {trial}',
                        config={'trial': trial, 'group': group_name, 'train split': args.train_split})
        model = create_model()

        train_ds_sample = train_ds.shuffle(buffer_size=len(train_ds)).take(train_size).batch(batch_size)
        history = model.fit(
            train_ds_sample, validation_data=val_ds, verbose=0,
            callbacks=create_callbacks(), **fit_kwargs)

        eval_dict = eval_model(model, test_ds)
        wandb.log(eval_dict)
        wandb.finish(quiet=True)

        del model, train_ds_sample

def eval_model(model, test_ds):
    eval_metrics = model.evaluate(test_ds, return_dict=True, verbose=0)
    for split, ds in eval_datasets.items():
        ds_metrics = {f'{split}_{metric_name}': metric
            for metric_name, metric in model.evaluate(ds, return_dict=True, verbose=0).items()}
        eval_metrics = {**eval_metrics, **ds_metrics}

    return eval_metrics

# endregion

In [7]:
from models import get_obj_selector, get_normalizer, CNNEmbedder, cnn_embedder_kwargs, hidden_dense_size
from relational_neural_networks.relational_graphlet_convolution import RelationalGraphletConvolutionGroupAttn

relconv_kwargs = dict(n_filters=16, graphlet_size=3, n_groups=8, mdipr_kwargs=dict(rel_dim=16, proj_dim=4, symmetric=True),
    group_attn_key_dim=16, group_attn_key='pos', symmetric_inner_prod=False, permutation_aggregator='mean',
    filter_initializer='random_normal', beta=None, group_normalizer='softmax')
def create_relconvnet_groupattn(normalizer=None, freeze_embedder=False, object_selection=None):

    object_selector = get_obj_selector(object_selection)
    normalizer = get_normalizer(normalizer)
    cnn_embedder = CNNEmbedder(**cnn_embedder_kwargs)
    cnn_embedder.trainable = not freeze_embedder

    rel_conv = RelationalGraphletConvolutionGroupAttn(
        **relconv_kwargs, name='rgc')

    model = tf.keras.Sequential([
        object_selector,
        cnn_embedder,
        normalizer,
        rel_conv,
        tf.keras.layers.Flatten(name='flatten'),
        tf.keras.layers.Dense(hidden_dense_size, activation='relu', name='hidden_dense1'),
        tf.keras.layers.Dense(2, name='output')
        ], name='relconv'
    )

    return model

In [8]:

#region train & evaluate model

if args.object_selection:
    object_selection = models.get_obj_selection_by_task(args.task)
else:
    object_selection = None

def create_model():
    model = create_relconvnet_groupattn(args.normalizer, args.freeze_embedder, object_selection)
    model.compile(loss=loss, optimizer=create_opt(), metrics=metrics) # compile
    model.build(input_shape=(None, *train_ds.element_spec[0].shape)) # build
    return model

In [9]:
model = create_model()
model.summary()

Model: "relconv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 identity_obj_selector (Ide  (None, 9, 12, 12, 3)      0         
 ntity)                                                          
                                                                 
 cnn_embedder (CNNEmbedder)  (None, 9, 16)             2768      
                                                                 
 identity_normalizer (Ident  (None, 9, 16)             0         
 ity)                                                            
                                                                 
 rgc (RelationalGraphletCon  (None, 8, 16)             3920      
 volutionGroupAttn)                                              
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                           

In [10]:

group_name = models.get_group_name(args.model, args.normalizer, args.freeze_embedder, object_selection)

utils.print_section("TRAINING & EVALUATING MODEL")
train_model(
    create_model, eval_model, fit_kwargs, create_callbacks,
    wandb_project_name, group_name,
    train_ds, val_ds, test_ds,
    train_size, start_trial, num_trials
    )
#endregion



TRAINING & EVALUATING MODEL




trial:   0%|          | 0/1 [00:00<?, ?it/s]

NotFoundError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/awni/miniconda3/envs/attention/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_